# Verificacion de Estimaciones para la Campaña 2023.

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')

/var/folders/6c/17qbm4nd5318hd4bcrx68wjh0000gn/T/ipykernel_43049/1758670802.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')


In [4]:
df.columns

Index(['Ruta', 'Sitio', 'Año', 'Camara', 'Extra', 'Archivo', 'Fecha', 'Hora',
       'Animal_proba', 'Animal', 'Guanaco_proba', 'Guanaco', 'Especie',
       'Cantidad_proba', 'Cantidad', 'Validar', 'Validado'],
      dtype='object')

## Estimacion de Proporciones:

In [5]:
# Proporcion de Imagenes con Animales
pAnimal = sum(df['Animal']==1)/len(df)
print('Proporcion de Imagenes con Animales: ', round(pAnimal * 100,2), '%')

Proporcion de Imagenes con Animales:  83.45 %


In [6]:
# Proporcion de Imagenes sin Animales
pNoAnimal = sum(df['Animal']==0)/len(df)
print('Proporcion de Imagenes sin Animales: ', round(pNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales:  16.55 %


## Tiempo Total del Proceso:

### Tiempo de Ejecucion Automatica:

La eficiencia en los tiempos se vio perjudica por el uso de AWS para almacenar las imagenes. Al tener mayor latencia el proceso de consultar las imagenes demora más. Ademas, para consultar las imagenes se debieron tensorificar y extraer los metadatos antes de aplicar los modelos.

Se utilizaron multiprocesos concurrentes para hacer el proceso mas rapido. Y tecnicas de autoguardado para que en caso de que se rompa el proceso no se tenga que iniciar desde cero.

- Tensorificacion: 1.3 imagenes por segundo, por lo que el proceso completo llevo 25.4 horas.
- Extraccion de Metadata: 2.39 imagenes por segundo, por lo que el proceso completo llevo 13.8 horas.
- Modelos: XXXX


In [7]:
tTensorifizacion = 1.3
tMetadata = 2.39
tModelos = 6.25 # CAMBIAR
tAutom =(1/tTensorifizacion)+(1/tMetadata)+(1/tModelos)
TAutom = (len(df) * (tAutom))/(60*60)
print('Tiempo de Automatizacion: ', round(TAutom,2), 'horas')
print('Tiempo de Automatizacion: ', math.ceil(TAutom/8), 'jornadas')

Tiempo de Automatizacion:  44.46 horas
Tiempo de Automatizacion:  6 jornadas


Por lo que el tiempo total del proceso automatico es de **XX horas**  que son equivalentes a **XX jornadas**.

### Tiempo de Validacion:

El porcantaje de imagenes a validar sobre el total de la base es de:

In [26]:
pValNoAnimal = sum((df['Validar']==1) & (df['Animal']==0))/sum(df['Animal']==0)
print('Proporcion de Imagenes sin Animales a Validar: ', round(pValNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales a Validar:  55.1 %


In [27]:
pValAnimal = sum((df['Validar']==1) & (df['Animal']==1))/sum(df['Animal']==1)
print('Proporcion de Imagenes con Animales a Validar: ', round(pValAnimal * 100,2), '%')

Proporcion de Imagenes con Animales a Validar:  93.34 %


In [20]:
# Porcentaje de Imagenes a Validar:
pVal = sum(df['Validar']==1)/len(df)
print('Porcentaje de Imagenes a Validar: ', round(pVal * 100,2), '%')

Porcentaje de Imagenes a Validar:  87.01 %


In [9]:
# Tiempo de Validacion:
tVal = pVal * 1.36
TVal = (len(df) * tVal)/(60*60)
print('Tiempo de Validacion: ', round(TVal,2), 'horas')
print('Tiempo de Validacion: ', math.ceil(TVal/8), 'jornadas')

Tiempo de Validacion:  39.04 horas
Tiempo de Validacion:  5 jornadas


Por lo que el tiempo total del proceso de validacion es de **XX horas**  que son equivalentes a **XX jornadas**.

### Tiempo Total

El tiempo total del proceso sera de:

In [41]:
TTotal = TAutom + TVal
print('Tiempo del Proceso: ', round(TTotal,2), 'horas')
print('Tiempo del Proceso: ', math.ceil(TTotal/8), 'jornadas')

Tiempo del Proceso:  83.5 horas
Tiempo del Proceso:  11 jornadas


El proceso completo requiere **XX jornadas laborales**  (XX horas). Cabe destacar que de estas solo **XX horas requieren atencion humana**

### Cantidad de Imagenes por Hora:

Para calcular la cantidad de imágenes que se procesaran por hora se tiene en cuenta tanto el tiempo que tarda en procesarlas los modelos predictivos cómo el tiempo que se tarda en validar aquellas que necesiten un refuerzo en su clasificación. 

In [11]:
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos')

cHora = 60*60/(tAutom + tVal)
print('Cantidad de Imagenes por Hora: ', math.ceil(cHora))

Tiempo de Automatizacion:  1.35 segundos
Cantidad de Imagenes por Hora:  1423


### Precision de Etiquetado:

In [12]:
eNoAnimal = pNoAnimal * (95.94/100) * (1.45/100)
print('Error de Imagenes sin Animales: ', round(eNoAnimal * 100,2), '%')

Error de Imagenes sin Animales:  0.23 %


In [13]:
eAnimal = pAnimal * (49.19/100) * ((1.41/100) + (41.79/100) * (5.22/100))
print('Error de Imagenes con Animales: ', round(eAnimal * 100,2), '%')

Error de Imagenes con Animales:  1.47 %


In [14]:
eAutomatico = eNoAnimal + eAnimal
print('Error Automatico: ', round(eAutomatico * 100,2), '%')

Error Automatico:  1.7 %


In [15]:
cErrores = len(df) * eAutomatico
print('Cantidad de Errores: ', math.ceil(cErrores))

Cantidad de Errores:  2025


### Costos del Proceso:

- Costo Falso Negativo: $300
- Costo Falso Positivo: $1000
- Costo Error en Especies: $1300
- Costo Error en Cantidad: $650

In [17]:
coErrorNoAnimal = (0.95/100) * 300
print('Costo de Error de Imagenes sin Animales: $', round(coErrorNoAnimal,2))

Costo de Error de Imagenes sin Animales: $ 2.85


In [18]:
coErrorAnimal = pAnimal * (49.19/100) * ((1.41/100) * 1300 + (41.79/100) * (5.22/100) * 650)
print('Costo de Error de Imagenes con Animales: $', round(coErrorAnimal,2))

Costo de Error de Imagenes con Animales: $ 13.35


In [19]:
coErrorAutomatico = coErrorNoAnimal + coErrorAnimal
print('Costo de Error Automatico: $', round(coErrorAutomatico,2))

Costo de Error Automatico: $ 16.2


In [33]:
coErrorVal = pAnimal * pValAnimal * ((1.41/100) * 300 + (41.79/100) * (5.22/100) * 650)/2 + pNoAnimal * pValNoAnimal * ((0.6/100)/2) * 1000
print('Costo de Error de Validacion: $', round(coErrorVal,2))

Costo de Error de Validacion: $ 7.44


In [30]:
coOperacionalVal = pVal * tVal * 10730/(8*60*60)
print('Costo Operacional de Validacion: $', round(coOperacionalVal,2))

Costo Operacional de Validacion: $ 0.38


In [31]:
coServicio = 1100000
print('Costo de Servicio: $', round(coServicio,2))

Costo de Servicio: $ 1100000


In [35]:
coTotal = len(df) * (coErrorAutomatico + coErrorVal + coOperacionalVal) + coServicio
print('Costo Total: $', round(coTotal,2))

Costo Total: $ 3953016.44


## KPIs

### Tiempo Total del Proceso

In [46]:
print('Tiempo Total: ')
print('- Clasificacion Manual: 152.75 horas')
print('- Solucion Semi-Automatica:', round(TTotal,2), 'horas')
print('Variacion Porcentual: ', round((TTotal - 152.75)/152.75 * 100,2), '%')

Tiempo Total: 
- Clasificacion Manual: 152.75 horas
- Solucion Semi-Automatica: 83.5 horas
Variacion Porcentual:  -45.34 %


In [44]:
print('Jornadas Laborales:')
print('- Clasificacion Manual: 19.1 jornadas')
print('- Solucion Semi-Automatica:')
print('   - Jornadas Ocupadas: ', round(TVal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TVal/8 - 19.1)/19.1 * 100,2), '%')
print('    - Jornadas Totales: ', round(TTotal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TTotal/8 - 19.1)/19.1 * 100,2), '%')

Jornadas Laborales:
- Clasificacion Manual: 19.1 jornadas
- Solucion Semi-Automatica:
   - Jornadas Ocupadas:  4.9 jornadas
    Variacion Porcentual:  -74.45 %
    - Jornadas Totales:  10.4 jornadas
    Variacion Porcentual:  -45.36 %


### Cantidad de Imagenes Analizadas por Hora:

In [48]:
print('Cantidad de Imagenes por Hora: ')
print('- Clasificacion Manual: 777 imagenes/hr')
print('- Solucion Semi-Automatica: ', math.ceil(cHora), 'imagenes/hr')
print('Variacion Porcentual: ', round((cHora - 777)/777 * 100,2), '%')

Cantidad de Imagenes por Hora: 
- Clasificacion Manual: 777 imagenes/hr
- Solucion Semi-Automatica:  1423 imagenes/hr
Variacion Porcentual:  83.06 %


### Costo del Proceso:

In [50]:
print('Costo del Proceso:')
print('- Clasificacion Manual: $ 6722325.4')
print('- Solucion Semi-Automatica: $', round(coTotal,2))
print('Variacion Porcentual: ', round((coTotal - 6722325.4)/6722325.4 * 100,2), '%')

Costo del Proceso:
- Clasificacion Manual: $ 6722325.4
- Solucion Semi-Automatica: $ 3953016.44
Variacion Porcentual:  -41.2 %


### Ratio de Imagenes Utiles:

In [51]:
print('Ratio de Imagenes Utiles: ')
print('- Clasificacion Manual: 0.79%')
print('- Solucion Semi-Automatica: 100%')
print('Variacion Porcentual: ', round((100 - 0.79)/0.79 * 100,2), '%')

Ratio de Imagenes Utiles: 
- Clasificacion Manual: 0.79%
- Solucion Semi-Automatica: 100%
Variacion Porcentual:  12558.23 %
